<a href="https://colab.research.google.com/github/Venkatpotla33/ADS-Lab/blob/main/ADS_Lab_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Red Black Trees**

In [ ]:
# Minimal Red-Black Tree Implementation (Insert + Search + Inorder)

class Node:
    def __init__(self, key, color='R', left=None, right=None, parent=None):
        self.key, self.color, self.left, self.right, self.parent = key, color, left, right, parent

class RBTree:
    def __init__(self):
        self.NIL = Node(None, 'B')
        self.root = self.NIL

    def left_rotate(self, x):
        y = x.right
        x.right, y.left = y.left, x
        if y.left != self.NIL: y.left.parent = x
        y.parent = x.parent
        if x.parent == self.NIL: self.root = y
        elif x == x.parent.left: x.parent.left = y
        else: x.parent.right = y
        x.parent = y

    def right_rotate(self, y):
        x = y.left
        y.left, x.right = x.right, y
        if x.right != self.NIL: x.right.parent = y
        x.parent = y.parent
        if y.parent == self.NIL: self.root = x
        elif y == y.parent.right: y.parent.right = x
        else: y.parent.left = x
        y.parent = x

    def insert(self, key):
        z = Node(key, 'R', self.NIL, self.NIL)
        y, x = self.NIL, self.root
        while x != self.NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y == self.NIL: self.root = z
        elif z.key < y.key: y.left = z
        else: y.right = z
        self.fix_insert(z)

    def fix_insert(self, z):
        while z.parent.color == 'R':
            if z.parent == z.parent.parent.left:
                y = z.parent.parent.right
                if y.color == 'R': z.parent.color = y.color = 'B'; z.parent.parent.color = 'R'; z = z.parent.parent
                else:
                    if z == z.parent.right: z = z.parent; self.left_rotate(z)
                    z.parent.color, z.parent.parent.color = 'B', 'R'
                    self.right_rotate(z.parent.parent)
            else:
                y = z.parent.parent.left
                if y.color == 'R': z.parent.color = y.color = 'B'; z.parent.parent.color = 'R'; z = z.parent.parent
                else:
                    if z == z.parent.left: z = z.parent; self.right_rotate(z)
                    z.parent.color, z.parent.parent.color = 'B', 'R'
                    self.left_rotate(z.parent.parent)
        self.root.color = 'B'

    def search(self, key):
        x = self.root
        while x != self.NIL and x.key != key:
            x = x.left if key < x.key else x.right
        return x != self.NIL

    def inorder(self, node=None):
        if node is None: node = self.root
        if node == self.NIL: return []
        return self.inorder(node.left) + [node.key] + self.inorder(node.right)

# Example Usage
if __name__ == "__main__":
    rbt = RBTree()
    for n in [10, 20, 30, 15, 25, 5]:
        rbt.insert(n)
    print("Inorder traversal:", rbt.inorder())
    print("Search 15:", rbt.search(15))
    print("Search 50:", rbt.search(50))


# **B Trees**

In [8]:
class Node:
    def __init__(self, t, leaf=False):
        self.t, self.leaf = t, leaf
        self.keys, self.child = [], []

class BTree:
    def __init__(self, t): self.root, self.t = Node(t, True), t

    def search(self, k, x=None):
        x = x or self.root
        i = 0
        while i < len(x.keys) and k > x.keys[i]: i += 1
        if i < len(x.keys) and k == x.keys[i]: return True
        return False if x.leaf else self.search(k, x.child[i])

    def insert(self, k):
        r = self.root
        if len(r.keys) == 2*self.t-1:
            s = Node(self.t)
            self.root, s.child = s, [r]
            self.split(s,0)
            self._insert(s,k)
        else: self._insert(r,k)

    def _insert(self, x, k):
        i = len(x.keys)-1
        if x.leaf:
            x.keys.append(k)
            x.keys.sort()
        else:
            while i>=0 and k < x.keys[i]: i-=1
            i+=1
            if len(x.child[i].keys)==2*self.t-1:
                self.split(x,i)
                if k>x.keys[i]: i+=1
            self._insert(x.child[i],k)

    def split(self, x, i):
        t = self.t
        y = x.child[i]
        z = Node(t, y.leaf)
        x.child.insert(i+1,z)
        x.keys.insert(i,y.keys[t-1])
        z.keys, y.keys = y.keys[t:], y.keys[:t-1]
        if not y.leaf:
            z.child, y.child = y.child[t:], y.child[:t]

    def traverse(self, x=None):
        x = x or self.root
        for i in range(len(x.keys)):
            if not x.leaf: self.traverse(x.child[i])
            print(x.keys[i], end=' ')
        if not x.leaf: self.traverse(x.child[len(x.keys)])

# Example
if __name__=="__main__":
    b=BTree(3)
    for i in [10,20,5,6,12,30,7,17]: b.insert(i)
    b.traverse(); print()
    print("Search 6:", b.search(6))
    print("Search 15:", b.search(15))

5 6 7 10 12 17 20 30 
Search 6: True
Search 15: False
